### NEURAL NETWORKS

Neural networks can be constructed using the torch.nn package.

Now that you had a glimpse of autograd, nn depends on autograd to define models and differentiate them. An nn.Module contains layers, and a method forward(input)that returns the output.

For example, look at this network that classifies digit images:

![Image](https://pytorch.org/tutorials/_images/mnist.png)

It is a simple feed-forward network. It takes the input, feeds it through several layers one after the other, and then finally gives the output.

A typical training procedure for a neural network is as follows:

- Define the neural network that has some learnable parameters (or weights)
- Iterate over a dataset of inputs
- Process input through the network
- Compute the loss (how far is the output from being correct)
- Propagate gradients back into the network’s parameters
- Update the weights of the network, typically using a simple update rule: weight = weight - learning_rate * gradient

#### Define the network

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # An affine operation: y = Wx + b
        self.fc1 = nn.Linear(16*6*6, 120) # 6x6 from image dimension.
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number.
        x = F.max_pool2d(F.relu(self.conv2(x)), 2) # as same as (2, 2)
        x = x.view(-1, self.num_flat_features(x)) # resize the tensor
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x
    
    def num_flat_features(self, x):
        
        size = x.size()[1:] # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [4]:
net = Net()
print(net)

params = list(net.parameters())
print(len(params))
print(params[0].size()) # conv1's weight

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)
10
torch.Size([6, 1, 3, 3])


In [5]:
# Let's try a random 32x32 input. Note: expected input sieze of this net(LeNet) is 32x32.
# To use this net on the MNIST dataset, place resize the images from the dataset to 32x32.

input_x = torch.randn(1, 1, 32, 32)
print(input_x)
out = net(input_x)
print(out)

tensor([[[[-0.9521,  0.1592, -0.9383,  ..., -0.0141, -0.6524,  0.7924],
          [-1.1102,  0.2138,  1.2086,  ...,  0.6870, -0.5910, -2.3196],
          [ 0.3988, -0.8289, -0.1150,  ...,  0.0791, -0.3271,  0.0649],
          ...,
          [ 0.2507,  0.8729, -0.6259,  ...,  0.1836,  1.1520,  0.2082],
          [-0.5946,  0.2594, -0.0212,  ...,  1.0993, -0.8617, -0.8530],
          [ 0.2239, -0.1706, -0.9999,  ...,  0.3826,  0.1268, -0.8152]]]])
tensor([[-0.0615,  0.1176, -0.0005,  0.1038, -0.1272, -0.0374,  0.0506, -0.1035,
         -0.1368, -0.0491]], grad_fn=<AddmmBackward>)


In [6]:
# Zero the gradient buffers of all parameters and backprops with random gradients:

net.zero_grad()
out.backward(torch.randn(1, 10))

- **torch.Tensor** - A multi-dimensional array with support for autograd operations like backward(). Also holds the gradient w.r.t. the tensor.
- **nn.Module** - Neural network module. Convenient way of encapsulating parameters, with helpers for moving them to GPU, exporting, loading, etc.
- **nn.Parameter** - A kind of Tensor, that is automatically registered as a parameter when assigned as an attribute to a Module.
- **autograd.Function** - Implements forward and backward definitions of an autograd operation. Every Tensor operation creates at least a single Function node that connects to functions that created a Tensor and encodes its history.

#### Loss Function

A loss function takes the (output, target) pair of inputs, and computes a value that estimates how far away the output is from the target.

There are several different loss functions under the nn package . A simple loss is: nn.MSELoss which computes the mean-squared error between the input and the target.

For example:

In [8]:
output = net(input_x)
target = torch.randn(10)
target = target.view(1, -1)
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.3783, grad_fn=<MseLossBackward>)


Now, if you follow loss in the backward direction, using its .grad_fn attribute, you will see a graph of computations that looks like this:

```python
input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
      -> view -> linear -> relu -> linear -> relu -> linear
      -> MSELoss
      -> loss
```

#### Backprop

To backpropagate the error all we have to do is to loss.backward(). You need to clear the existing gradients though, else gradients will be accumulated to existing gradients.

Now we shall call loss.backward(), and have a look at conv1’s bias gradients before and after the backward.

In [9]:
net.zero_grad() # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-3.0236e-03, -4.9244e-04,  5.0710e-03, -4.8239e-05,  6.1012e-03,
         1.2590e-03])


#### Update the weights

The simplest update rule used in practice is the Stochastic Gradient Descent (SGD):

    weight = weight - learning_rate * gradient
    
We can implement this using simple Python code:

In [10]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

However, as you use neural networks, you want to use various different update rules such as SGD, Nesterov-SGD, Adam, RMSProp, etc. To enable this, we built a small package: torch.optim that implements all these methods. Using it is very simple:

In [12]:
import torch.optim as optim

# Create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# In your training loop:
optimizer.zero_grad()
output = net(input_x)
loss = criterion(output, target)
loss.backward()
optimizer.step()